In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import time
import datetime
import math
import prawcore

In [ ]:
my_client_id = ''
my_client_secret = ''
my_user_agent = ''

import praw
from psaw import PushshiftAPI
reddit = praw.Reddit(client_id=my_client_id, client_secret=my_client_secret, user_agent=my_user_agent)
api = PushshiftAPI(reddit)

Subreddit is commented out to search the whole of reddit

In [ ]:
def getPushshiftData(query, after, before):#, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=10000&after='+str(after)+'&before='+str(before)#+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [ ]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    upvote = subm['upvote_ratio']
    
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair,upvote))
    subStats[sub_id] = subData

In [ ]:
#Subreddit to query
#sub='conspiracy'
#before and after dates
before = "1596743277" #13/07
#before = math.ceil(datetime.utcnow().timestamp())
after = "1483228800"  #08/07 
query = "catalonia"
subCount = 0
subStats = {}

In [ ]:
data = getPushshiftData(query, after, before)#, sub)# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before)#, sub)
    
print(len(data))

In [ ]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

In [ ]:
def updateSubs_file():
    upload_count = 0
    location = "C:\\Users\\posit\\Downloads"
    print("input filename of submission file, please add .csv")
    filename = input()
    file = location + filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post_ID","Title","Url","Author","Score","Publish_Date","Total_No_of_Comments","Permalink","Flair","Upvote"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
        
updateSubs_file()

In [ ]:
submissions = []
comments = []

In [ ]:
for i in df['Post_ID']:
    print(i)
    submission = reddit.submission(id=f'{i}')
    submissions.append(submission)
    try:
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            comments.append(comment)
    except prawcore.exceptions.Forbidden:
        continue
    except prawcore.exceptions.NotFound:
        continue

In [ ]:
len(comments)

In [ ]:
author = []
body = []
date = []
link_id = []
parent_id = []
sub_com = []
score_com = []
r_com = []
replies = []
date_rep = []
author_rep = []
link_rep = []
parent_rep = []
sub_rep = []
score_rep = []
r_rep = []

for i in range(len(comments)):
    author.append(comments[i].author)
    body.append(comments[i].body)
    date.append(comments[i].created_utc)
    link_id.append(comments[i].link_id)
    parent_id.append(comments[i].parent())
    sub_com.append(comments[i].submission)
    score_com.append(comments[i].score)
    r_com.append(comments[i].subreddit)
    for reply in comments[i].replies:
        replies.append(reply.body)
        date_rep.append(reply.created_utc)
        author_rep.append(reply.author)
        link_rep.append(reply.link_id)
        parent_rep.append(reply.parent())
        sub_rep.append(reply.submission)
        score_rep.append(reply.score)
        r_rep.append(reply.subreddit)

In [ ]:
author_df = pd.DataFrame(author)
author_df['index'] = author_df.index
comment_df = pd.DataFrame(body)
comment_df['index'] = comment_df.index
date_df = pd.DataFrame(date)
date_df['index'] = date_df.index
link_df = pd.DataFrame(link_id)
link_df['index'] = link_df.index
parent_df = pd.DataFrame(parent_id)
parent_df['index'] = parent_df.index
sub_df = pd.DataFrame(sub_com)
sub_df['index'] = sub_df.index
score_df = pd.DataFrame(score_com)
score_df['index'] = score_df.index
r_df = pd.DataFrame(r_com)
r_df['index'] = r_df.index

In [ ]:
author_rep_df = pd.DataFrame(author_rep)
author_rep_df['index'] = author_rep_df.index
comment_rep_df = pd.DataFrame(replies)
comment_rep_df['index'] = comment_rep_df.index
date_rep_df = pd.DataFrame(date_rep)
date_rep_df['index'] = date_rep_df.index
link_rep_df = pd.DataFrame(link_rep)
link_rep_df['index'] = link_rep_df.index
parent_rep_df = pd.DataFrame(parent_rep)
parent_rep_df['index'] = parent_rep_df.index
sub_rep_df = pd.DataFrame(sub_rep)
sub_rep_df['index'] = sub_rep_df.index
score_rep_df = pd.DataFrame(score_rep)
score_rep_df['index'] = score_rep_df.index
r_rep_df = pd.DataFrame(r_rep)
r_rep_df['index'] = r_rep_df.index

In [ ]:
comments_df = pd.DataFrame()
comments_df['index'] = np.arange(0,181182,1)

In [ ]:
comments_df = comments_df.join(author_df,on='index',how='left',rsuffix='_right')
comments_df = comments_df.join(comment_df,on='index',how='left',rsuffix='_right')
comments_df = comments_df.join(date_df,on='index',how='left',rsuffix='_right')
comments_df = comments_df.join(link_df,on='index',how='left',rsuffix='_right')
comments_df = comments_df.join(parent_df,on='index',how='left',rsuffix='_right')
comments_df = comments_df.join(sub_df,on='index',how='left',rsuffix='_right')
comments_df = comments_df.join(score_df,on='index',how='left',rsuffix='_right')
comments_df = comments_df.join(r_df,on='index',how='left',rsuffix='_right')

In [ ]:
comments_df.columns

In [ ]:
comments_df.drop(['index_right','index'], axis=1,inplace=True)
comments_df.columns = ['author', 'comment','date','link','parent','submission','score','subreddit']
comments_df.head()

In [ ]:
len(replies)

In [ ]:
replies_df = pd.DataFrame()
replies_df['index'] = np.arange(0,149624,1)

In [ ]:
replies_df = replies_df.join(author_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(comment_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(date_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(link_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(parent_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(sub_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(score_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(r_rep_df,on='index',how='left',rsuffix='_right')

In [ ]:
replies_df.columns

In [ ]:
replies_df.drop(['index_right','index'], axis=1,inplace=True)
replies_df.columns = ['author', 'comment','date','link','parent','submission','score','subreddit']
replies_df.head()

In [ ]:
all_com_rep = pd.DataFrame()
all_com_rep = comments_df.append(replies_df)
all_com_rep

In [ ]:
all_com_rep.to_csv("Raddit.csv",index=False)
all_com_rep = pd.read_csv('C:\\Users\\posit\\Reddit.csv')

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect("DB.db")

In [ ]:
df.columns

In [ ]:
all_com_rep.columns

In [ ]:
c = conn.cursor()

def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS Submissions(Post_ID TEXT, Title TEXT,Url TEXT, Author TEXT, Score REAL,Publish_Date TEXT, Total_No_of_Comments REAL, Permalink TEXT, Flair TEXT)")

create_table()

In [ ]:
def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS Comments(author TEXT, comment TEXT,date INTEGER, link TEXT, parent REAL,submission TEXT, score REAL, subreddit TEXT)")

create_table()

In [ ]:
df.to_sql("Submissions", conn, if_exists = 'append', index = False)
all_com_rep.to_sql("Comments", conn, if_exists = 'append', index = False)